# v2 Rebalancing API Example

This is a replication of Binu's Postman Env and Collection.

In [1]:
import pandas as pd
import json
import requests

### Step 1: Setup Request Environment
Reading postman environment provided by Binu

In [2]:
with open('V2 Rebalance API.postman_environment.json', 'r') as env_json:
    env_data = json.load(env_json)

env = {}
for row in env_data.get("values"):
    env[row.get('key')] = row.get('value')

env

{'admin-apiKey': 'JUF^fqRW7ao9X&SY9^n%BGc',
 'URL': 'https://fsalgoapims.azure-api.net/',
 'apiKey': '6177321eac992341d1ad0823a07e76bfc4ee6909db120e377ea303fdc216756c',
 'Ocp-Apim-Subscription-Key': 'fa061f8aeb454788ac54dfaea685fdaa',
 'azureUrl': 'https://fsalgoapims.azure-api.net'}

### Step 2: Get Portfolio Rebalance Data
Again, from Binu's PostMan config but could be the API request body from CPS Reports

In [3]:
coll_file = 'V2 Rebalancing API.postman_collection.json'
with open(coll_file, 'r') as req_json:
    req_data = json.load(req_json)

rebal_req_body = dict(json.loads(req_data.get('item')[0].get('request').get('body').get('raw')))
rebal_req_body.keys()

dict_keys(['ExchangeRates', 'HierarchicalModelPortfolio', 'InvestmentDetails', 'PortfolioHoldings', 'PortfolioMandateRules', 'RequestOptions', 'SimpleModelPortfolios'])

In [4]:
rebal_req_body.get('PortfolioHoldings').get('Accounts')[0]

{'AccountID': 'ACC1',
 'AccountName': '503',
 'AdvancedOptions': {'AccountType': '66c08dc0-4871-48cf-9f62-bd72a060119f',
  'BuyPriority': 0,
  'SellPriority': 0,
  'CashAdjustmentCalculationCurrencyValue': -50000},
 'CashHoldings': [{'Balance': 10000,
   'CashAccountName': 'USD CASH',
   'CurrencyCode': 'USD',
   'Reserved': 0},
  {'Balance': 20000,
   'CashAccountName': 'AUD CASH',
   'CurrencyCode': 'AUD',
   'Reserved': 0}],
 'Investments': [{'InvestmentId': '9db84185-16d8-4610-a215-04057ee361d4',
   'Parcels': [{'CostBase': 2433.53,
     'ParcelId': 1,
     'PurchaseDate': '2024-12-11T00:00:00',
     'PurchasePrice': 2433.53,
     'Quantity': 10,
     'Reserved': 0}]},
  {'InvestmentId': '0b3366c4-06fa-4101-a5dc-f6d2613a5258',
   'Parcels': [{'CostBase': 5.63,
     'ParcelId': 1,
     'PurchaseDate': '2024-12-11T00:00:00',
     'PurchasePrice': 5.63,
     'Quantity': 5000,
     'Reserved': 0}]},
  {'InvestmentId': '0c3ba234-06d4-4a5b-9f4a-abbfb813a583',
   'Parcels': [{'CostBase': 

### Step 3: Make the Request
With the body and auth tokens, we can now send the data to the rebal engine

In [6]:
headers = {
    'Content-Type': 'application/json',
    'x-fsee-apikey': env.get('apiKey'),
    'Ocp-Apim-Subscription-Key': env.get('Ocp-Apim-Subscription-Key')
}

rebal_url = 'https://fsalgoapims.azure-api.net/rebalance'
response = requests.post(rebal_url, json=rebal_req_body, headers=headers)

response

<Response [200]>

Once we have the response data, we can access the data as a dictionary and do a variety of wonderful pythonic things with it, such as look at the keys

In [7]:
rebal_data = response.json()
rebal_data.keys()

dict_keys(['responseId', 'requestId', 'clientRequestId', 'calculationCurrencyMovement', 'cashAccountsMovements', 'investmentHoldingsMovements', 'errors', 'diagnostics'])

We could access all the data or we could slice it and look at something that is more interesting to analysts.

In the below example we will look at the `investmentHoldingsMovements`

In [8]:
rebal_data.get('investmentHoldingsMovements')

[{'investmentId': '9db84185-16d8-4610-a215-04057ee361d4',
  'investmentCode': 'GOOG',
  'marketCode': 'NASDAQ',
  'price': 2000,
  'investmentCurrency': 'USD',
  'purchasePrice': 0,
  'costBase': 2112.396667,
  'realisedGain': 0,
  'unrealisedGain': 0,
  'investmentMovement': {'type': 'Sell',
   'units': -30,
   'value': 87000.0,
   'brokerage': 0,
   'parcelOrders': [{'accountId': 'ACC1',
     'parcelId': 1,
     'purchaseDate': '2024-12-11T00:00:00',
     'purchasePrice': 2433.53,
     'costBase': 2433.53,
     'realisedGain': None,
     'unrealisedGain': None,
     'orderType': 'Sell',
     'orderUnits': -10,
     'orderValue': 29000.0,
     'orderBrokerage': 0.0,
     'currentUnits': 10,
     'currentValue': 29000.0,
     'currentPercentage': 0.119959,
     'finalUnits': 0,
     'finalValue': 0.0,
     'finalPercentage': 0,
     'reservedUnits': None},
    {'accountId': 'ACC3',
     'parcelId': 1,
     'purchaseDate': '2025-01-20T00:00:00',
     'purchasePrice': 2000,
     'costBas

What we will mostly find is that analysts will want to use the `Pandas` library to look at the data in Table.

In [9]:
rebal_df = pd.DataFrame(rebal_data.get('investmentHoldingsMovements'))
rebal_df

,investmentId,investmentCode,marketCode,price,investmentCurrency,purchasePrice,costBase,realisedGain,unrealisedGain,investmentMovement,targetUnits,targetValue,targetPercentage,currentUnits,currentValue,currentPercentage,finalUnits,finalValue,finalPercentage,reservedUnits
0,9db84185-16d8-4610-a215-04057ee361d4,GOOG,NASDAQ,2000,USD,0,2112.396667,0,0,"{'type': 'Sell', 'units': -30, 'value': 87000....",0,0.0,0.0,30,87000.0,0.359876,0,0.0,0.000000,None
1,0b3366c4-06fa-4101-a5dc-f6d2613a5258,360,ASX,5,AUD,0,5.630000,0,0,"{'type': 'Buy', 'units': 10340, 'value': -5170...",19340,96700.0,0.4,9000,45000.0,0.186143,19340,96700.0,0.400000,None
2,0c3ba234-06d4-4a5b-9f4a-abbfb813a583,MSFT,NASDAQ,250,USD,0,250.000000,0,0,"{'type': 'Buy', 'units': -7, 'value': 2537.5, ...",133,48350.0,0.2,140,50750.0,0.209928,133,48212.5,0.199431,None
3,844a328b-9863-4dc0-8129-78f59f8754f8,TNE,ASX,10,AUD,0,10.000000,0,0,"{'type': 'Buy', 'units': 3252, 'value': -32520...",7252,72525.0,0.3,4000,40000.0,0.165460,7252,72520.0,0.299979,None
